# Imports

In [ ]:
#@title <font size="5">← ឵឵<i>Upgrade FFmpeg to v4.2.2</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('pip install mediapipe')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
os.system('apt install libvidstab-dev')
clear_output()
print('Installation finished.')

Installation finished.


In [1]:
from google.colab import drive, auth
drive.mount('/content/drive')
auth.authenticate_user()

Mounted at /content/drive


In [2]:
import glob
import json
import os
import re
import subprocess

from collections import defaultdict

import cv2
import gspread as gs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from oauth2client.client import GoogleCredentials
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
gc = gs.authorize(GoogleCredentials.get_application_default())

path = '/content/drive/MyDrive/ML_boulder/videos/' 

In [4]:
%cd /content/drive/MyDrive/ML_boulder/videos/

/content/drive/.shortcut-targets-by-id/18QGGslm5AdAiiRMVe7lKIfIUl8x3SLK9/ML_boulder/videos


# Preprocessing

## Stabilization

In [5]:
#Functions to find unprocessed vids
def find_rest_vids(path):
  res = []
  for up in os.listdir(path):
    if re.search(r'boulder_[5-6]_.*$', up):
      for inside in os.listdir(up):
        if len(inside) > 0 and os.path.isdir(f'{up}/{inside}'):
          for v in os.listdir(f'{up}/{inside}'):
            if re.search(r'(^IMG_.*.MOV$)', v):
              res.append(f'{up}/{inside}/{v}')

  return res

def find_rest_mp4(path):
  res = []
  for up in os.listdir(path):
    if re.search(r'boulder_[1-6]_.*$', up):
      for inside in os.listdir(up):
        if len(inside) > 0 and os.path.isdir(f'{up}/{inside}'):
          for v in os.listdir(f'{up}/{inside}'):
            if re.search(r'(^MOV_.*.mp4$)|((^.*.MP4$))', v):
              res.append(f'{up}/{inside}/{v}')

  return res  

def find_stab_vids(path):
  res = []
  for up in os.listdir(path):
    if re.search(r'boulder_[1-6]_.*$', up):
      for inside in os.listdir(up):
        if len(inside) > 0 and os.path.isdir(f'{up}/{inside}'):
          for v in os.listdir(f'{up}/{inside}'):
            if re.search(r'(^.*_STAB.MOV$)', v):
              res.append(f'{up}/{inside}/{v}')

  return res

def find_all_vids(path):
  res = []
  for up in os.listdir(path):
    if re.search(r'boulder_[1-6]_.*$', up):
      for inside in os.listdir(up):
        if len(inside) > 0 and os.path.isdir(f'{up}/{inside}'):
          for v in os.listdir(f'{up}/{inside}'):
            if re.search(r'(^.*.MOV$)|(^MOV_.*.mp4$)|(^.*.MP4$)', v):
              res.append(f'{up}/{inside}/{v}')

  return res

def find_unstab_vids(path):
  all_vids = find_all_vids(path)
  stab_vids = find_stab_vids(path)
  non_stab_vids = list(set(all_vids) - set(stab_vids))
  left_vids = [v[:-9] for v in list(set([f'{v}_STAB.MOV' for v in non_stab_vids]) - set(stab_vids))]
  if len(left_vids) == 0:
    print("All done, no videos to stabilize !")
  return left_vids

In [6]:
#Used for stabilizing all the videos in a given list of paths (by default the still unprocessed vids)
def stabilize(vid_list=None):
  if vid_list is None:
    vid_list = find_unstab_vids(path)
    
  for v in vid_list:
    transform_command = ['ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', f'{v}', '-vf', f'vidstabdetect=result={v}.trf', '-f', 'null', '-']
    stab_command = ['ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', f'{v}', '-vf', f'vidstabtransform=input={v}.trf:smoothing=0', f'{v}_STAB.MOV']
    
    print(f"*** Processing {v}...")
    if subprocess.run(transform_command).returncode == 0:
      if subprocess.run(stab_command).returncode == 0:
        print ("All good, stabilized version saved")
      else:
        print ("There was an error running the stabilisation script")
    else:
      print ("There was an error running the transform script")

## Time cropping

In [7]:
# Used for reading the excel sheet
def get_sheet(i):
  worksheet = gc.open('boulder_problems').get_worksheet(i)

  # get_all_values gives a list of rows.
  rows = worksheet.get_all_values()

  # Convert to a DataFrame and render.
  df = pd.DataFrame.from_records(rows)
  df.columns = df.iloc[0] 
  df = df[1:]
  return df

In [8]:
def crop_time(df):
  for index, vid in df.iterrows():
    p_in = 'ml/good/' + vid.Folder + vid.File
    dir = 'videos/' + vid.Folder
    p_out = dir + vid.File

    subprocess.run(['mkdir', '-p', dir])

    if(vid.time_start != '-1' and vid.time_end != '-1'):
      command = ['ffmpeg', '-y', '-ss', vid.time_start, '-i', p_in,  '-to', vid.time_end, '-c', 'copy', p_out]
      subprocess.run(command)
    elif(vid.time_start == '-1' and vid.time_end != '-1'):
      command = ['ffmpeg', '-y', '-i', p_in,  '-to', vid.time_end, '-c', 'copy', p_out]
      subprocess.run(command)
    elif(vid.time_start != '-1' and vid.time_end == '-1'):
      command = ['ffmpeg', '-y', '-ss', vid.time_start, '-i', p_in, '-c', 'copy', p_out]
      subprocess.run(command)
    else:
      command = ['ffmpeg', '-y', '-i', p_in, '-c', 'copy', p_out]
      subprocess.run(command)

In [9]:
def time_crop_all():
  for i in range(5):
    df = get_sheet(i) 
    crop_time(df)

## Screengrabs

In [10]:
# To grab screen shots from mediapipe processed videos
mediapipe = True
# Takes the last frame for the videos that didn't work
redo = True

In [11]:
def take_screen(df):
  for index, vid in df.iterrows():
    if mediapipe:
      p_in = 'videos/' + vid.Folder + vid.File + '_POSE.mp4'
    else:
      p_in = 'videos/' + vid.Folder + vid.File

    if not os.path.isfile(f'{p_in}_SCREEN.jpg'):
      if vid.time_screenshot != '' and not redo:
        command = ['ffmpeg', '-ss', vid.time_screenshot, '-i', p_in, '-vframes', '1', '-q:v', '2', f'{p_in}_SCREEN.jpg']
      else:
        command = ['ffmpeg', '-sseof', '-2', '-i', p_in, '-update', '1', '-q:v', '2', f'{p_in}_SCREEN.jpg']
      if verbose == True:
        print(subprocess.run(command))

In [12]:
def screen_all():
  for i in range(5):
    df = get_sheet(i) 
    take_screen(df)

# Pose estimation

In [13]:
left_hand = ["LEFT_THUMB.x", "LEFT_INDEX.x", "LEFT_PINKY.x",  "LEFT_WRIST.x", "LEFT_THUMB.y", "LEFT_INDEX.y", "LEFT_PINKY.y",  "LEFT_WRIST.y", "LEFT_THUMB.v", "LEFT_INDEX.v", "LEFT_PINKY.v",  "LEFT_WRIST.v"]
right_hand = ["RIGHT_THUMB.x", "RIGHT_INDEX.x", "RIGHT_PINKY.x",  "RIGHT_WRIST.x", "RIGHT_THUMB.y", "RIGHT_INDEX.y", "RIGHT_PINKY.y",  "RIGHT_WRIST.y", "RIGHT_THUMB.v", "RIGHT_INDEX.v", "RIGHT_PINKY.v",  "RIGHT_WRIST.v"]
left_foot = ["LEFT_FOOT_INDEX.x", "LEFT_ANKLE.x", "LEFT_HEEL.x", "LEFT_FOOT_INDEX.y", "LEFT_ANKLE.y", "LEFT_HEEL.y", "LEFT_FOOT_INDEX.v", "LEFT_ANKLE.v", "LEFT_HEEL.v"]
right_foot = ["RIGHT_FOOT_INDEX.x", "RIGHT_ANKLE.x", "RIGHT_HEEL.x", "RIGHT_FOOT_INDEX.y", "RIGHT_ANKLE.y", "RIGHT_HEEL.y", "RIGHT_FOOT_INDEX.v", "RIGHT_ANKLE.v", "RIGHT_HEEL.v"]
extremities = {"left_hand" : left_hand, "right_hand": right_hand, "left_foot" : left_foot, "right_foot" : right_foot}

In [14]:
def landmark_to_dict(results):
  dict_ = { "LEFT_FOOT_INDEX": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                                "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y,
                                "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].visibility},
            "LEFT_ANKLE": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility},
            "LEFT_HEEL": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].visibility},
            "RIGHT_FOOT_INDEX": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x,
                                "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y,
                                 "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].visibility},
            "RIGHT_ANKLE": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility},
            "RIGHT_HEEL": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].visibility},
            "LEFT_THUMB": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_THUMB].x,
                            "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_THUMB].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_THUMB].visibility},
            "LEFT_INDEX": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility},
            "LEFT_PINKY": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_PINKY].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_PINKY].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_PINKY].visibility},
            "LEFT_WRIST": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].visibility},
            "RIGHT_THUMB": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_THUMB].x,
                            "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_THUMB].y,
                            "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_THUMB].visibility},
            "RIGHT_INDEX": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility},
            "RIGHT_PINKY": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_PINKY].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_PINKY].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_PINKY].visibility},
            "RIGHT_WRIST": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].visibility}, 
            "NOSE": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                     "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                     "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility}, 
            "RIGHT_ELBOW": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                            "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y,
                            "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].visibility}, 
            "LEFT_ELBOW": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                            "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].visibility},  
            "RIGHT_KNEE": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x,
                            "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y,
                           "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].visibility}, 
            "LEFT_KNEE": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y,
                          "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].visibility}, 
           "RIGHT_HIP": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y,
                         "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].visibility}, 
            "LEFT_HIP": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x,
                          "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y,
                         "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].visibility},
           "RIGHT_SHOULDER": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                              "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                              "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility}, 
            "LEFT_SHOULDER": {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                              "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                              "v": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility},   
      }
  return dict_

In [15]:
# Utils
def coord(height, width, side_h, frac_h, side_w, frac_w):
  frac_h = frac_h / 100
  frac_w = frac_w / 100

  left = 0
  right = width - 1
  top = 0
  bottom = height - 1


  if(side_h == 'center'):
    top = round((1 - frac_h) * height / 2)
    bottom = height - 1 - top
  elif(side_h == 'top'):
    bottom = height - 1 - round((1 - frac_h) * height)
  elif(side_h == 'bottom'):
    top = round((1 - frac_h) * height)
  

  if(side_w == 'center'):
    left = round((1 - frac_w) * width / 2)
    right = width - 1 - left
  elif(side_w == 'left'):
    right = width - 1 - round((1 - frac_w) * width)
  elif(side_w == 'right'):
    left = round((1 - frac_w) * width)
  

  return top, bottom, left, right

def crop_image(img, left, right, bottom, top):
  return img[top:bottom, left:right]

In [16]:
def estimate_pose(df, output_video = True):
  mp_pose = mp.solutions.pose
  mp_drawing = mp.solutions.drawing_utils 
  mp_drawing_styles = mp.solutions.drawing_styles

  for index, vid in df.iterrows():
    cap = cv2.VideoCapture('/content/drive/MyDrive/ML_boulder/videos/' + vid.Folder + vid.File)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    top, bottom, left, right = coord(frame_height, frame_width, vid.crop_y_side, vid.crop_y, vid.crop_x_side, vid.crop_x)
    width = right - left
    height = bottom - top
    
    data=[]

    #command = ['mkdir', '-p', '/content/drive/MyDrive/ML_boulder/videos/' + f'{vid.Folder}']
    #subprocess.run(command)

    if output_video:
      out = cv2.VideoWriter('/content/drive/MyDrive/ML_boulder/videos/' + vid.Folder + vid.File + '_POSE.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (width, height))

    with mp_pose.Pose() as pose:
      while True:
        # Convert the BGR image to RGB and process it with MediaPipe Pose.
        success, image = cap.read()

        if not success:
          break

        # Crop the image
        image = crop_image(image, left, right, bottom, top)

        imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(imgRGB)

        #Change here to add empty dict if no detection ?
        if not results.pose_landmarks:
          continue

        landmark = landmark_to_dict(results)
        data.append(landmark)
        
        if output_video:
          annotated_image = image.copy()
          mp_drawing.draw_landmarks(
              annotated_image,
              results.pose_landmarks,
              mp_pose.POSE_CONNECTIONS,
              landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
          
          out.write(annotated_image)

    with open('/content/drive/MyDrive/ML_boulder/videos/' + vid.Folder + vid.File + '_POSE.json', 'w') as write_file:
      json.dump(data, write_file)

    if output_video:
      out.release()

    cap.release()
    if verbose == True:
      print(vid.File)

In [17]:
def estimate_all(output_video = False):
  for i in range(5):
    df = get_sheet(i) 
    estimate_pose(df, output_video)

# Move sequence

In [18]:
def weighted_m(x, w):
  sum_w = w.sum()
  num = np.multiply(x,w).sum()
  return (num/sum_w)

def check_hold_helper(nb_static_frame, index, df, threshold):
  df_frame = df.iloc[index:(index + nb_static_frame), :]
  nb_ldmk = int(df_frame.shape[1]/3)
  mean_x = np.empty([nb_static_frame, 1 ])
  mean_y = np.empty([nb_static_frame, 1 ])
  for i in range(nb_static_frame):
    visi = np.array(df_frame.iloc[i,2*nb_ldmk:])
    coord_x = np.array(df_frame.iloc[i,:(nb_ldmk)])
    coord_y = np.array(df_frame.iloc[i,(nb_ldmk):(2*nb_ldmk)])
    mean_x[i] = weighted_m(coord_x, visi)
    mean_y[i] = weighted_m(coord_y, visi)
  x = float(mean_x[0])
  y = float(mean_y[0])
  mean_x = np.abs(mean_x - mean_x[0])
  mean_y = np.abs(mean_y - mean_y[0])
  if (mean_x < threshold).all() and (mean_y < threshold).all():
    return(x, y, index)

def check_hold(body_part, data_frame):
  df_sub = data_frame[body_part]
  nb_static_f = 30
  threshold = 0.02
  res = []
  for i in range(df_sub.shape[0] - nb_static_f):
    coord = check_hold_helper(nb_static_f, i, df_sub, threshold)
    (res.append(coord) if coord is not None else None)
  return res

In [19]:
def seq(df, extrem_dict):
  result = defaultdict(list)
  for key, part in extrem_dict.items():
    res = check_hold(part, df)
    res_arr = np.array([[a[0], a[1]] for a in res])

    silhouettes = []
    if len(res_arr) > 0:
      for k in range(2, min(len(res_arr), 12)):
        clustering = KMeans(n_clusters=k).fit_predict(res_arr)
        score = silhouette_score(res, clustering)
        silhouettes.append({"k" : k, "score": score})

      silhouettes = pd.DataFrame(silhouettes)
      n = silhouettes.k[silhouettes.score.argmax()]

      cluster_f = KMeans(n_clusters=n).fit(res_arr)
      centers = cluster_f.cluster_centers_

      for id, cl in enumerate(cluster_f.labels_):
        if cluster_f.labels_[id - 1] != cl:
          result[key].append((centers[cl], res[id][2]))
  return result

In [20]:
def display_seq(img, centers):
  frame_width = img.shape[1]
  frame_height = img.shape[0]
  left_hand_color = (0, 204, 204)
  right_hand_color = (0, 204, 0)
  left_foot_color = (255, 102, 102)
  right_foot_color = (76, 0, 153)

  size = 30
  thickness = 4

  center_inorder = []

  for key, coord in centers.items():
    for elem in coord:
      x = int(elem[0][0] * frame_width)
      y = int(elem[0][1] * frame_height)
      if key == 'left_foot':
        center_inorder.append((x , y, elem[1], left_foot_color))
      elif key == 'right_foot':
        center_inorder.append((x , y, elem[1], right_foot_color))
      elif key == 'left_hand':
        center_inorder.append((x , y, elem[1], left_hand_color))
      else:
        center_inorder.append((x , y, elem[1], right_hand_color))
  
  center_inorder.sort(key = lambda x: x[2])

  for id, elem in  enumerate(center_inorder):
    x_bg = elem[0] - size
    y_bg = elem[1] - size
    x_ed = elem[0] + size
    y_ed= elem[1] + size

    cv2.rectangle(img, (x_bg, y_bg), (x_ed, y_ed), elem[3], thickness)

    cv2.putText(img, "Move:{}".format(id), (x_bg, y_bg - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, elem[3], 2)

  cv2.putText(img, "Left_hand", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, left_hand_color, 3)
  #cv2.circle(img, (150, 30), radius=8, color = left_hand_color, thickness=-1)
  cv2.putText(img, "Right_hand", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, right_hand_color, 3)
  #cv2.circle(img, (40, 80), radius=8, color = right_hand_color, thickness=-1)
  cv2.putText(img, "Left_foot", (20, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.9, left_foot_color, 3)
  #cv2.circle(img, (40, 130), radius=8, color = left_foot_color, thickness=-1)
  cv2.putText(img, "Right_foot", (20, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.9, right_foot_color, 3)

In [ ]:
def create_gif_arrow(img, centers):
  frames = []
  timeframe = []

  frames.append(img)

  frame_width = img.shape[1]
  frame_height = img.shape[0]

  scaled_centers = copy.deepcopy(centers)

  move_order = []

  for key, coord in scaled_centers.items():
    for elem in coord:
      elem[0][0] *= frame_width
      elem[0][1] *= frame_height
      move_order.append((key, elem[1]))

  move_order.sort(key=lambda y: y[1])

  members = ['left_hand', 'right_hand', 'left_foot', 'right_foot']
  colors = {'left_hand': (0, 204, 204), 'right_hand':(0, 204, 0), 'left_foot': (255, 102, 102), 'right_foot':(76, 0, 153)}

  img = add_legend(img, colors, members)

  size = 30
  thickness = 4
  
  img_base = img.copy()

  count = dict(zip(members, [[0,0], [0,0], [0,0], [0,0]]))

  for id, move in enumerate(move_order):
    img = img_base.copy()

    for member in members:
      start = count[member][0]
      end = count[member][1]
      draw_last_moves(img, scaled_centers[member][start:end], colors[member])

    member = move[0]

    count[member][1] += 1 #add 1 to the end
    start = count[member][0] #get start and end
    end = count[member][1]
    draw_last_moves(img, scaled_centers[member][end-1:end], colors[member])


    if (count[member][1] > 1):
      draw_arrow(img, scaled_centers[member][start:end], colors[member])
      count[member][0] = count[member][1] - 1 #

    frames.append(img)
    timeframe.append(move[1])

  return frames, timeframe

def add_legend(img, colors, members):
  text_pos = [(20, 30), (20, 80), (20, 130), (20, 180)]
  for i in range(4):
    cv2.putText(img, members[i], text_pos[i], cv2.FONT_HERSHEY_SIMPLEX, 0.9, colors[members[i]], 3)
  return img

def draw_arrow(img, last_move, color):
  size = 30
  thickness = 4

  x1 = int(last_move[0][0][0])
  y1 = int(last_move[0][0][1] - size)

  x2 = int(last_move[1][0][0])
  y2 = int(last_move[1][0][1] + size)

  cv2.arrowedLine(img, (x1, y1), (x2, y2), color, thickness)

  return img

def draw_last_moves(img, last_move, color):
  size = 30
  thickness = 4
  for id, elem in  enumerate(last_move):
    x_bg = int(elem[0][0] - size)
    y_bg = int(elem[0][1] - size)
    x_ed = int(elem[0][0] + size)
    y_ed = int(elem[0][1] + size)
    cv2.rectangle(img, (x_bg, y_bg), (x_ed, y_ed), color, thickness)
  return img

def save_gif(img, dict_centers, p_out):
  frames, timeframe = create_gif_arrow(img, dict_centers)
  timeframe_scaled = [timeframe[i] - timeframe[i-1] for i in range(1,len(timeframe))] #get number of frame btween two moves
  timeframe_scaled = [timeframe_scaled[i] if timeframe_scaled[i] < 60 else 60 for i in range(len(timeframe_scaled))]#cap the maximum duration bteween two move to two seconds
  timeframe_scaled.insert(0, 30) #add boulder image without any move, at the beginning, for 1 sec
  timeframe_scaled.append(60) #stay on last image for 2 sec
  timeframe_scaled = list(np.divide(timeframe_scaled,30)) #divide by 30 to have speed of 0.5 (original 60fps)
  logging.debug(f"Saving GIF file for {p_out}")
  with imageio.get_writer(p_out, mode="I", duration = timeframe_scaled) as writer:
      for idx, frame in enumerate(frames):
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          writer.append_data(rgb_frame)
          logging.debug(f"Adding frame {idx+1} to GIF for {p_out}: ")

In [21]:
def process_sheet(sheet, gif):
  for index, vid in sheet.iterrows():
    v = path + vid.Folder + vid.File
    p_in = v + '_POSE.json'
    p_out = v + '_MOVE_SEQ.jpg'
    if not os.path.isfile(p_out) or redo_move_seq:
      if verbose:
        print(f"Calculating move sequence for {v}...")

      data =json.load(open(p_in))
      df = pd.json_normalize(data)
      if very_verbose:
        print(f"Loaded dataframe for {v}...")

      img = cv2.imread(v + '_POSE.mp4_SCREEN.jpg')

      dict_centers = seq(df, extremities)
      if very_verbose:
        print(f"Calculated centers for {v}...")

      display_seq(img, dict_centers)
      if very_verbose:
        print(f"Displaying move sequence for {v}...")
      cv2.imwrite(p_out, img)
      if verbose:
        print(f"Saved {p_out}.")
      if gif:
        save_gif(img, dict_centers, f'{v}_MOVE_SEQ.gif')

In [22]:
def process_all(gif):
  for i in range(4):
    sheet = get_sheet(i) 
    process_sheet(sheet, gif)

# Putting it all together

In [23]:
verbose = True
very_verbose = True

preprocessing = False
stab = False
crop = False
screen = False

pose_estimation = False
output_video = False

move_sequence = True
redo_move_seq = False

gif = True

In [24]:
if preprocessing:
  print("Preprocessing videos...")
  if stab:
    print("Stabilizing videos...")
    stabilize()
    print("Stabilization done !")
  if crop:
    print("Cropping videos...")
    time_crop_all()
    print("Cropping done !")
  if screen:
    print("Taking screenshots...")
    screen_all()
    print("Screenshots done !")
  print("Preprocessing done !")
else:
  print("Preprocessing skipped !")

if pose_estimation:
  print("Beginning pose estimation...")
  estimate_all(output_video)
  print("Pose estimation done !")
else:
  print("Pose estimation skipped !")

if move_sequence:
  print("Move sequence generation...")
  process_all(gif)    
  print("Move sequence generation done !")
else:
  print("Move sequence generation skipped !")

print("All done !")

Preprocessing skipped !
Pose estimation skipped !
Move sequence generation...
Calculating move sequence for /content/drive/MyDrive/ML_boulder/videos/boulder_6_25/Fail/IMG_6853.MOV_STAB.MOV...
Loaded dataframe for /content/drive/MyDrive/ML_boulder/videos/boulder_6_25/Fail/IMG_6853.MOV_STAB.MOV...
Calculated centers for /content/drive/MyDrive/ML_boulder/videos/boulder_6_25/Fail/IMG_6853.MOV_STAB.MOV...
Displaying move sequence for /content/drive/MyDrive/ML_boulder/videos/boulder_6_25/Fail/IMG_6853.MOV_STAB.MOV...
Saved /content/drive/MyDrive/ML_boulder/videos/boulder_6_25/Fail/IMG_6853.MOV_STAB.MOV_MOVE_SEQ.jpg.
Calculating move sequence for /content/drive/MyDrive/ML_boulder/videos/boulder_6_26/Fail/IMG_6857.MOV_STAB.MOV...
Loaded dataframe for /content/drive/MyDrive/ML_boulder/videos/boulder_6_26/Fail/IMG_6857.MOV_STAB.MOV...
Calculated centers for /content/drive/MyDrive/ML_boulder/videos/boulder_6_26/Fail/IMG_6857.MOV_STAB.MOV...
Displaying move sequence for /content/drive/MyDrive/ML_b